# Shipping and Conflict

This notebook explores the relationship between trade and conflict in the Red Sea region. The focus is on the Beb El-Mandeb strait and the conflict in Yemen. 

In [48]:
import pandas as pd

In [49]:
df = pd.read_csv("../../data/portwatch/chokepoints_data_02_26_2024.csv")
bab_el_mandeb = df[
    (df["portname"] == "Bab el-Mandeb Strait") & (df["date"] > "2022-12-26")
]

In [50]:
bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])

C:\Users\sahit\AppData\Local\Temp\ipykernel_22968\3150530413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bab_el_mandeb["date"] = pd.to_datetime(bab_el_mandeb["date"])


In [51]:
bab_el_mandeb = (
    bab_el_mandeb.groupby(pd.Grouper(key="date", freq="W"))["n_total"]
    .sum()
    .reset_index()
)  # merge(conflict_by_country, on = 'date')

In [52]:
conflict_by_country = pd.read_csv(
    "../../data/conflict/conflict_by_country_2023-01-01_2024-02-27.csv"
)
conflict_by_port = pd.read_csv(
    "../../data/conflict/conflict_by_port_2023-01-01_2024-02-27.csv"
)

In [53]:
conflict_by_country["event_date"] = pd.to_datetime(conflict_by_country["event_date"])
conflict_by_country.rename(columns={"event_date": "date"}, inplace=True)

In [54]:
conflict_by_port["event_date"] = pd.to_datetime(conflict_by_port["event_date"])
conflict_by_port.rename(columns={"event_date": "date"}, inplace=True)

In [55]:
conflict_by_country.drop(columns="Unnamed: 0", inplace=True)
conflict_by_port.drop(columns="Unnamed: 0", inplace=True)

In [56]:
al_ahmadi = (
    conflict_by_port[conflict_by_port["portname"] == "Al Ahmadi"]
    .groupby(["portname", "date"])[["nrEvents", "nrFatalities"]]
    .sum()
    .reset_index()
)

In [57]:
data = (
    conflict_by_country[conflict_by_country["country"] == "Yemen"]
    .merge(bab_el_mandeb, on="date")
    .merge(al_ahmadi, on="date")
)
data.rename(
    columns={"nrEvents_x": "countryEvents", "nrEvents_y": "portEvents", "nrFatalities_x":"countryFatalities", "nrFatalities_y":"portFatalities"}, inplace=True
)

In [58]:
from datetime import datetime

conflict_date = datetime(2023, 10, 7)
crisis_date = datetime(2023, 11, 17)

In [59]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Legend, Span

# Call this to display plots inline in a Jupyter Notebook
output_notebook()

data["date"] = pd.to_datetime(data["date"])


# Create a figure
p = figure(
    x_axis_type="datetime",
    plot_width=1000,
    plot_height=400,
    title="Comparing Weekly Trends in Conflict and Shipping",
)
p.title.text_font_size = "16pt"

# Plot multiple lines
p1 = p.line(data["date"], data["n_total"], line_width=2, color="#B07AA1")
p2 = p.line(data["date"], data["countryFatalities"], line_width=2, color="#FF9DA7")
p3 = p.line(data["date"], data["portFatalities"], line_width=2, color="#9C755F")

legend_items = [
    ("Number of Vessels in Bal El-Mandab Strait", [p1]),
    ("Number of fatalities in Yemen", [p2]),
    ("Number of fatalities around Port Al Ahmadi", [p3]),
]

# Create the legend manually
legend = Legend(items=legend_items, location="center")

# Add the legend to the plot, but ensure it's outside the main figure
p.add_layout(legend, "right")

# Customize the plot
p.yaxis.axis_label = "Number of Vessels and Number of Conflict Fatalities"
p.legend.location = "top_right"
p.yaxis.axis_label_text_font_style = "normal"

p.add_layout(
    Span(
        location=crisis_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)
p.add_layout(
    Span(
        location=conflict_date,
        dimension="height",
        line_color="#7C7C7C",
        line_width=2,
        line_dash=(4, 4),
    )
)

# Show the plot
show(p)

Loading BokehJS ...

### Exploring Correlation between Weekly Trends of Conflict Fatalities in Yemen and Shipping Vessels in Bab El-Mandab

In [60]:
acled = pd.read_csv("../../data/conflict/acled_raw_2023-01-01_2024-02-27.csv")
acled["event_date"] = pd.to_datetime(acled["event_date"])
acled = acled.groupby(["country", "event_date"])["fatalities"].sum().reset_index()

In [61]:
acled.rename(columns={"fatalities": "countryFatalities", "event_date": "ymd"}, inplace=True)

In [62]:
df["ymd"] = pd.to_datetime(df["ymd"])

In [63]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Div
from bokeh.layouts import column
import numpy as np


# Enable notebook output
output_notebook()

title_main = "Correlation between Weekly Yemen Conflict Fatalities and Shipping Vessels"
subtitle = "Conflict Fatalities ocurred in all of Yemen, Vessels are traversing Bal El-Mandab Strait"
source_text = "Source: Conflict Fatalities from ACLED; Shipping Vessels from WB estimates from IMF PortWatch"


# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(data)

slope, intercept = np.polyfit(data["countryFatalities"], data["n_total"], 1)
x = np.array(data["countryFatalities"])
y_line = slope * x + intercept

# Create a ColumnDataSource for the line of best fit
line_source = ColumnDataSource(data=dict(x=x, y_line=y_line))

# Initialize Bokeh figure
p = figure(plot_width=800, plot_height=400)

# p.title.text_font_size = '12pt'
# p.title.text_font_style = "normal"
# p.title.align = 'left'

# Add scatter plot glyphs
p.circle(
    x="countryFatalities", y="n_total", source=source, size=10, color="navy", alpha=0.5
)

p.line("x", "y_line", source=line_source, color="grey", line_width=2, line_dash=(4, 4))

# Customize plot
p.yaxis.axis_label = "Total Vessels traversing the Bab El-Mandab Strait"
p.xaxis.axis_label = (
    "Reported Conflict Fatalities in Yemen between Jan 1 2023 and Feb 27 2024"
)

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

title_fig = Div(
    text=f"<h3 style='text-align:left; font-size:16pt;margin:0'>{title_main}</h3>",
    width=800,
    height=40,
    margin=(-10, 0, 0, 20),
)
title_fig2 = Div(
    text=f"<h3 style='text-align:left; font-size:12pt;font-weight:normal;margin:0'>{subtitle}</h3>",
    width=800,
    height=40,
    margin=(10, 0, 0, 20),
)
sub_title_fig = Div(
    text=f"<h3 style='text-align:left; font-size:10pt;font-style:normal; font-weight:normal;margin:0'>{source_text}</h3>",
    width=800,
    height=30,
)

layout = column(title_fig, title_fig2, p, sub_title_fig)

# Show plot
show(layout)

Loading BokehJS ...

The Correlation Coefficient between the **Yemen Weekly Conflict Fatalities** and **Weekly Vessels in Bab El-Mandab** is 0.0572


In [64]:
np.corrcoef(data["countryFatalities"].values, data["n_total"].values)[0, 1]

0.05721067865633851

In [65]:
np.corrcoef(data["portFatalities"].values, data["n_total"].values)[0, 1]

0.10279575331864839

In [66]:
daily_data = df[df['portid']=='chokepoint4'].merge(acled[acled['country']=='Yemen'], on='ymd')

In [67]:
np.corrcoef(daily_data["countryFatalities"].values, daily_data["n_total"].values)[0, 1]

0.02900569955715835

### Exploring Correlation between Daily Trends of Conflict Fatalities in Yemen and Shipping Vessels in Bab El-Mandab

In [68]:
daily_data = df[df["portid"] == "chokepoint4"].merge(
    acled[acled["country"] == "Yemen"], on="ymd"
)

In [69]:
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
import numpy as np

# Enable notebook output
output_notebook()

title_main = "Correlation between Daily Conflict Fatalities and Shipping Vessels"
subtitle = "Conflict Fatalities ocurred in a all of Yemen, Vessels are traversing Bal El-Mandab Strait"
source_text = "Source: Conflict Fatalities from ACLED; Shipping Vessels from WB estimates from IMF PortWatch"

# Create a ColumnDataSource from the DataFrame
source = ColumnDataSource(daily_data)

slope, intercept = np.polyfit(daily_data["countryFatalities"], daily_data["n_total"], 1)
x = np.array(daily_data["countryFatalities"])
y_line = slope * x + intercept

# Create a ColumnDataSource for the line of best fit
line_source = ColumnDataSource(data=dict(x=x, y_line=y_line))

# Initialize Bokeh figure
p = figure(plot_width=750, plot_height=400)

# Add scatter plot glyphs
p.circle(
    x="countryFatalities", y="n_total", source=source, size=10, color="navy", alpha=0.5
)

p.line("x", "y_line", source=line_source, color="grey", line_width=2, line_dash=(4, 4))

# Customize plot
p.yaxis.axis_label = "Total Vessels traversing Bab El-Mandab Strait"
p.xaxis.axis_label = (
    "Reported Conflict Fatalities in Yemen between Jan 1 2023 and Feb 27 2024"
)

p.xaxis.axis_label_text_font_style = "normal"
p.yaxis.axis_label_text_font_style = "normal"

title_fig = Div(
    text=f"<h3 style='text-align:left; font-size:16pt;margin:0'>{title_main}</h3>",
    width=800,
    height=40,
    margin=(-10, 0, 0, 20),
)
title_fig2 = Div(
    text=f"<h3 style='text-align:left; font-size:12pt;font-weight:normal;margin:0'>{subtitle}</h3>",
    width=800,
    height=40,
    margin=(10, 0, 0, 20),
)
sub_title_fig = Div(
    text=f"<h3 style='text-align:left; font-style:normal; font-size:10pt;font-weight:normal;margin:0'>{source_text}</h3>",
    width=800,
    height=30,
)

layout = column(title_fig, title_fig2, p, sub_title_fig)

# Show plot
show(layout)

Loading BokehJS ...

The Correlation Coeffecient between the **Yemen Daily Conflict Fatalities** and **Daily Vessels in Bab El-Mandab** is 0.029